## Imports

In [7]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex
import GetOldTweets3 as got
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
import datetime


pd.set_option("max_rows", 75)


## Function to Pull Tweets

In [17]:
# This function uses Reem's GOT3 code to import tweets given query, date range and number of tweets and saves it to a csv file

def get_tweets(query,start='2006-03-21',end=datetime.date.today().strftime("%Y-%m-%d"),maxtweets=1000):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                                               .setSince(start)\
                                               .setUntil(end)\
                                               .setMaxTweets(maxtweets)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)

    tweet_dict = tweetCriteria.__dict__
    file_name = (tweet_dict['querySearch']+\
                 tweet_dict['since']+\
                 tweet_dict['until']).replace(" ","_")   
    df = pd.DataFrame([t.__dict__ for t in tweet])
    return df.to_csv(f'../data/{query}_{maxtweets}.csv', index=False)

## Set Disaster/Search Info

In [14]:
# input search words and date range to pull tweets
search = 'hurricanemaria'
start_date = '2017-08-01'
end_date = '2017-10-15'
tweet_count = 10000


### Run function to pull tweets with above parameters and save as CSV

In [18]:
# pull tweets with keyword search, optional: start and end dates and maxtweets to pull(default is 1000)
get_tweets(search,start=start_date, end= end_date, maxtweets=tweet_count)

### Import CSV back into Jupyter as a DataFrame

In [ ]:
# pull csv of tweets back in as a dataframe for parsing
tweets = pd.read_csv(f'../data/{query}/{search}_{tweet_count}.csv')

# drop rows with NaN in the text column
tweets.dropna(axis=0,subset = ['text'],inplace=True)
tweets.shape

In [273]:
retweets = pd.read_csv(f'./data/{search}_retweet.csv')

In [274]:
retweets.shape

(10000, 15)

## RETWEETS

The GetOldTweets pull doesn't show non original tweets, instead it represents them in a column called retweets. So the idea is we take the original tweet and add the number of retweet rows with the same information 

In [138]:
# take each tweet and multiply the row by the number of retweets as a weight for that tweet
tweets = pd.read_csv(f'./data/{search}_{tweet_count}.csv')
# drop any rows with empty tweets
tweets.dropna(axis=0,subset = ['text'],inplace=True)
print(f' original dataframe shape: {tweets.shape}')

FileNotFoundError: [Errno 2] File ./data/hurricaneharvey_10000.csv does not exist: './data/hurricaneharvey_10000.csv'

In [276]:
retweets.to_csv('./data/{search}_retweets.csv',index=False)

In [203]:

def retweets(df):
    for ind, row in df.iterrows():
    for rt in range(0,df['retweets'].iloc[ind]):
        retweets = retweets.append(pd.Series(row,index=test.columns))
    print(f' new shape with retweets duplicated: {retweets.shape}')
    return retweets

,Unnamed: 0,username,to,text,retweets,favorites,replies,id,permalink,author_id,date,formatted_date,hashtags,mentions,geo,urls
8616,8616,Jasamsdestiny,NaN,Just a quick FYI peeps. Ranch and River have m...,5,3,4,1023117759104860160,https://twitter.com/Jasamsdestiny/status/10231...,129877411,2018-07-28 08:07:37+00:00,Sat Jul 28 08:07:37 +0000 2018,#MendocinoComplex #RiverFire #Ranchfire #Mendo...,0,NaN,NaN


In [6]:
tweets = pd.read_csv('../workflow/data/hurricaneharvey/twitter_retrieval/hurricaneharvey_10000.csv')

## Get related Hashtags

In [145]:
# Function to get hashtags that are also used with initial hashtag
def new_hashes(df_series):
    hashtags=[]
    for ht in df_series:
        for h in str(ht).split(' #'):
            hashtags.append(h.replace('#',''))
    df = pd.DataFrame(hashtags, columns=['hashtag'])
    df['count'] = [1 for i in range(1,len(df)+1)]
    
    return df.groupby('hashtag').sum().sort_values(by='count',ascending = False)
    
df_hts = new_hashes(tweets['hashtags'])

In [151]:
df_hts.head()

,count
hashtag,
HurricanHarvey,9036
hurricanharvey,822
Houston,545
Texas,396
prayfortexas,299
